In [3]:
# Import required libraries
import pandas as pd
import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output
import plotly.express as px

In [5]:
# Read the airline data into pandas dataframe
URL = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"
spacex_df = pd.read_csv(URL)
spacex_df.head()

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [7]:


# Read the airline data into pandas dataframe

max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Generate a marks dictionary

marks_dict = {i: str(i) for i in range(0, 10000, 1000)}

# Generate Launch Site Options

launch_sites = spacex_df['Launch Site'].unique()
options = [{'label':site, 'value':site} for site in launch_sites]
options.insert(0, {'label': 'All Sites', 'value': 'All Sites'})

## Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40, 'padding':'5px 5px','background-color':'lightblue'}),
                                html.Label('Select a Launch Site Here:', style={'font-size':30, 'color':'blue', 'padding':'3px 3px', 'background-color':'lightyellow' }),  
                                        
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown',
                                options=options,                              
                                placeholder='Chose a launch site location',
                                value = 'All Sites',             
                                searchable=True,
                                style = {'font-size':20, 'background-color':'lightpink', 'padding':'2px 2px'},
                                ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                min=0,
                                max=10000,
                                step=1000,
                                marks=marks_dict,
                                value=[min_payload, max_payload]), 
                                

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback( Output(component_id='success-pie-chart', component_property='figure'),
               Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    
    if entered_site == 'All Sites':
        data =  spacex_df[spacex_df['class']==1]
        fig = px.pie(data,
                     values='class',
                     names='Launch Site',
                     title='Total Success Launches by Site'
                     )
    else:
        data = spacex_df[spacex_df['Launch Site'] ==entered_site]
        fig = px.pie(data,
                     names = 'class',
                     title='Total Success/Failure Rate per Launch Site {}'.format(entered_site))
    return(fig)

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback( Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'),
               Input(component_id='payload-slider',component_property='value')])

def get_scatter_slider_chart(entered_site, payload_slider):
    min_payload, max_payload = payload_slider
    if entered_site == 'All Sites':
        data = spacex_df[(spacex_df['Payload Mass (kg)'] >= min_payload) & (spacex_df['Payload Mass (kg)'] <= max_payload)]
        fig = px.scatter(data, 
        x = 'Payload Mass (kg)',
        y = 'class',
        color = 'Booster Version Category',
        title = 'Correlation Between Payload and Mission Outcomes For All Sites')
        return fig
    else:
        new_data = spacex_df[spacex_df['Launch Site'] == entered_site]
        fig = px.scatter(new_data[(new_data['Payload Mass (kg)'] >= min_payload) & (new_data['Payload Mass (kg)'] <= max_payload)], 
        x = 'Payload Mass (kg)',
        y = 'class',
        color = 'Booster Version Category',
        title = 'Correlation Between Payload And Mission Outcomes for: ' + entered_site)
        return fig


  
                          
                            
                            
                              
    
               

               
                            
# Run the app
if __name__ == '__main__':
    app.run()
    


After visual analysis using the dashboard, you should be able to obtain some insights to answer the following five questions:

   * Which site has the largest successful launches?
   * Which site has the highest launch success rate?
   * Which payload range(s) has the highest launch success rate?
   * Which payload range(s) has the lowest launch success rate?
   * Which F9 Booster version (v1.0, v1.1, FT, B4, B5, etc.) has the highest launch success rate?

Answers to the above questions are:

* The site with the largest number of successful launches is KSC LC-39A at a percentage of 41.7%
* The site with the highest launch success rate is CCAFS SLC-40 with a success rate of 76.9%
* The payload range between 2000kg and 4000kg had the most successful launches with  a total of 12 successful launches
* The payload range between 6000kg and 8000kg had the least successful launches with zero successful launches
* The F9 Booster version B5 has the highest success rate of 100% with 1 successful launch out of 1 launch! 




